# Notebook to plot 2D spatial PFT's for all the depths

In [1]:
# import os
import sys
import numpy as np
import pandas as pd
from netCDF4 import Dataset as NetCDFFile 
from datetime import datetime
import numpy as np 
import matplotlib.pyplot as plt


import os
os.environ["PROJ_LIB"] = '/opt/conda/share/proj'
from mpl_toolkits.basemap import Basemap

# from mpl_toolkits.basemap import Basemap
from numpy import ma
from matplotlib import ticker, cm
import matplotlib as mpl
import matplotlib.ticker as tick
import proplot as plot
from matplotlib.colors import LogNorm
import matplotlib.colors as mplc
from matplotlib.ticker import LogFormatter 
import warnings
warnings.filterwarnings('ignore') # Ignore warning messages for printing

## Add path of .nc files and path of .png files to be saved

In [2]:

DATE_TODAY = "02_12_2021" # This DATE_TODAY should be same as the first notebook

###############################################################################################
# # To run in home folder please uncomment below paths
plot_path='/'.join(['/home/jovyan/Phytoplankton_EOV/PFT_Product/Plots', DATE_TODAY])
path_TACMOB='/'.join(['/home/jovyan/Phytoplankton_EOV/PFT_Product/Outputs', DATE_TODAY])
###############################################################################################


###############################################################################################
# # To run in WorkSpace VRE folder please uncomment below paths
# plot_path='/'.join(['/workspace/VREFolders/Zoo-Phytoplankton_EOV/Phytoplankton_EOV/PFT_Product/Plots', DATE_TODAY])
# path_TACMOB='/'.join(['/workspace/VREFolders/Zoo-Phytoplankton_EOV/Phytoplankton_EOV/PFT_Product/Outputs', DATE_TODAY])
###############################################################################################

if not os.path.isdir(plot_path):
    os.mkdir(plot_path)

### Making all plots for year 2018
### Plots will be saved corresponding monthly folders

### All PFT's per each depth are in one plot

In [ ]:
##########################################################################
##########################################################################

# New Depths
CMEMS_pres_N=[0, 5, 10, 15, 20, 25, 30, 35, 40,45, 50, 55, 60, 65, 70, 80, 90, 100,
              125, 150, 175, 200, 225, 250, 275, 300, 350, 400, 450, 500, 550, 600,
              700, 800, 900, 1000]
# Year
y = 2018
MONTHS=[1,2,3,4,5,6,7,8,9,10,11,12]
# MONTHS = [1]
str_months = ['January', 'February', "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
for m in MONTHS:
    print(y)
    print(m)
    M = str(m).zfill(2)
    
    #Create the subdirectory of path_TACMOB where the NetCDF of 3D Chl will be stored
    dir_plot_year = '/'.join([plot_path,str(y)])
    if not os.path.isdir(dir_plot_year):
        os.mkdir(dir_plot_year)
        
    dir_plot_month = '/'.join([plot_path,str(y),str(m)])
    if not os.path.isdir(dir_plot_month):
        os.mkdir(dir_plot_month) 
    
    dir_plot=[plot_path,str(y),str(m),str('PFT_ALL')] # Change the folder name with Variable 
    path_png_plot ='/'.join(dir_plot)
    if not os.path.isdir(path_png_plot):
        os.mkdir(path_png_plot)
    dir_nc_data='/'.join([path_TACMOB,str(y),str(m)])
    path_nc_file=''.join([dir_nc_data,'/SOCA_PFT_glo_bgc3d_rep_',str(y),'_',str(M),'_P',str(DATE_TODAY),'.nc'])
    print(path_nc_file)
    nc_data = NetCDFFile(path_nc_file)
    Micro_3D = nc_data.variables['Micro'][:]
    Nano_3D = nc_data.variables['Nano'][:]
    Pico_3D = nc_data.variables['Pico'][:]    
    Total_pig_3D = Micro_3D + Nano_3D + Pico_3D
    
    lat = nc_data.variables['latitude'][:]
    lon = nc_data.variables['longitude'][:]
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    
    levels = np.logspace(-3,1,121)
    formatter = LogFormatter(base=10.0, labelOnlyBase=False, minor_thresholds=None, linthresh=None) 


    for ii in np.arange(len(CMEMS_pres_N)):
        plt.rcParams.update({'figure.max_open_warning': 0})
        fig = plt.figure(ii+1, figsize=(12, 6),facecolor='white') 
        fig.patch.set_facecolor('white')
        
        
        ax=fig.add_subplot(311)
        text =  "PFT's at {}m depth for {} {}"
        plt.text(-160, 100,text.format(str(CMEMS_pres_N[ii]),str_months[m-1],str(y), transform=ax.transAxes, fontsize=10))
        ax.set_ylabel('Latitude',fontsize=12, labelpad=25)
#         ax.set_xlabel('Longitude',fontsize=12,labelpad=25)
        m_map = Basemap(projection='cyl',llcrnrlat=-90,urcrnrlat=90, llcrnrlon=-180,urcrnrlon=180,resolution='c')
        m_map.drawcoastlines(linewidth=0.1)
        m_map.fillcontinents(color='grey',lake_color='white')
        m_map.drawparallels(np.arange(-90,90,15),labels=[1,0,0,1],latmax=90, linewidth=0.3, fontsize=7)
        m_map.drawmeridians(np.arange(-180,180,30),labels=[0,0,0,0], rotation=45, linewidth=0.3, fontsize=7)        
        cf=ax.contourf(lon_grid,lat_grid,Micro_3D[0,ii,:,:], cmap='jet', levels=levels, norm=mplc.LogNorm())        
        cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.01,ax.get_position().height])
        cb = fig.colorbar(cf,cax=cax, ticks=[0.001, 0.01, 0.1,1,10], format=formatter) 
        cb.ax.set_yticklabels(labels=[0.001, 0.01,0.1,1,10],fontsize=6)
        cb.set_label('Micro-Chla (mg $m^{-3}$)', rotation=90, fontsize=8)    

        
        ax=fig.add_subplot(312)
        ax.set_ylabel('Latitude',fontsize=12, labelpad=25)
#         ax.set_xlabel('Longitude',fontsize=12,labelpad=25)
        m_map = Basemap(projection='cyl',llcrnrlat=-90,urcrnrlat=90, llcrnrlon=-180,urcrnrlon=180,resolution='c')
        m_map.drawcoastlines(linewidth=0.1)
        m_map.fillcontinents(color='grey',lake_color='white')
        m_map.drawparallels(np.arange(-90,90,15),labels=[1,0,0,1],latmax=90, linewidth=0.3, fontsize=7)
        m_map.drawmeridians(np.arange(-180,180,30),labels=[0,0,0,0], rotation=45, linewidth=0.3, fontsize=7)  
        cf=ax.contourf(lon_grid,lat_grid,Nano_3D[0,ii,:,:], cmap='jet', levels=levels, norm=mplc.LogNorm())        
        cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.01,ax.get_position().height])
        cb = fig.colorbar(cf,cax=cax, ticks=[0.001, 0.01, 0.1,1,10], format=formatter) 
        cb.ax.set_yticklabels(labels=[0.001, 0.01,0.1,1,10],fontsize=6)
        cb.set_label('Nano-Chla (mg $m^{-3}$)', rotation=90, fontsize=8)      

        
        ax=fig.add_subplot(313)
        ax.set_ylabel('Latitude',fontsize=12, labelpad=25)
        ax.set_xlabel('Longitude',fontsize=12,labelpad=25)
        m_map = Basemap(projection='cyl',llcrnrlat=-90,urcrnrlat=90, llcrnrlon=-180,urcrnrlon=180,resolution='c')
        m_map.drawcoastlines(linewidth=0.1)
        m_map.fillcontinents(color='grey',lake_color='white')
        m_map.drawparallels(np.arange(-90,90,15),labels=[1,0,0,1],latmax=90, linewidth=0.3, fontsize=7)
        m_map.drawmeridians(np.arange(-180,180,30),labels=[1,0,0,1], rotation=45, linewidth=0.3, fontsize=7)  
        cf=ax.contourf(lon_grid,lat_grid,Pico_3D[0,ii,:,:], cmap='jet', levels=levels, norm=mplc.LogNorm())        
        cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.01,ax.get_position().height])
        cb = fig.colorbar(cf,cax=cax, ticks=[0.001, 0.01, 0.1,1,10], format=formatter) 
        cb.ax.set_yticklabels(labels=[0.001, 0.01,0.1,1,10],fontsize=6)
        cb.set_label('Pico-Chla (mg $m^{-3}$)', rotation=90, fontsize=8)      
     

        pp2 = ''.join([str(path_png_plot),'/SOCA_PFT_ONLY_MG_',str(y),'_',str_months[m-1],'_',str(CMEMS_pres_N[ii]),'m_depth.png'])
        fig.savefig(pp2, bbox_inches='tight', dpi=300)
        plt.close()
        fig.clear()

##########################################################################
##########################################################################